In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from pandarallel import pandarallel
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

import sys
sys.path.append('../')
from data_loaders.enzyme_dataloader import get_structure_sequence
from dataset_preprocess.pdb_preprocess_utils import map_active_site_for_one     # 这里的begain_zero=False，位点从1开始





/home/xiaoruiwang/software/miniconda3/envs/py38/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


### 处理由于amber最小化之后将氨基酸残基改变的问题，例如HIS的质子化

In [2]:
def get_mcsa_normal_dataset(dir, subset=['train', 'valid'], flag='mcsa', read_new=False):
    dataset = pd.DataFrame()
    for dataset_flag in subset:
        sub_df = pd.read_csv(os.path.join(dir, f'{dataset_flag}_dataset' if not read_new else f'new_{dataset_flag}_dataset', f'{flag}_{dataset_flag}.csv'))
        sub_df['dataset_flag'] = [dataset_flag for _ in range(len(sub_df))]
        dataset = pd.concat([dataset, sub_df])
    return dataset


def get_sequence(structure_dir, structure_id):
    structure_id = structure_id.replace(' ', '')
    abs_input_path = os.path.abspath(os.path.join(structure_dir, f'{structure_id}_minimized.pdb'))
    sequence_calculated = get_structure_sequence(abs_input_path)
    return sequence_calculated


def map_active_site_for_one_return_list(seqA, seqB, active_site_A, active_site_type_A, begain_zero=False):
    seqB_active_site_bin = map_active_site_for_one(seqA, seqB, active_site_A, active_site_type_A, begain_zero=begain_zero)
    site_labels = np.argwhere(seqB_active_site_bin!=0).tolist()
    return site_labels

    

In [3]:
aug_dir_flag ='mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123'
new_aug_save_path = f'../dataset/mcsa_fine_tune/{aug_dir_flag}/'

fixed_generated_structure_dir = f'../dataset/mcsa_fine_tune/{aug_dir_flag}/esmfold_generated_aug_structures'
end_aug_dataset = get_mcsa_normal_dataset(new_aug_save_path, subset=['train', 'valid', 'test'], flag='aug_mcsa', read_new=True)
end_aug_dataset

,reaction,ec,alphafolddb-id,aa_sequence,site_labels,site_types,cluster,ec_level1,dataset_flag
0,CC1C(=O)N(C)C1C.O|MSIQHFRVALIPFFAAFCLPVFAHPETL...,3.5.2.6,P62593,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,"[[68], [71], [128], [164], [232], [235]]",NaN,Cluster 18632,3,train
1,CC1C(=O)N(C)C1C.O|MSIQHFRVALIPFFAAFCLPVFAHPETL...,3.5.2.6,P62593-c0_m7,TRPEERFPMMSTMKRLLCGAVLSRWPEWEQLGRRICYEQGTLVESP...,"[[11], [14], [66], [99], [158], [161]]",NaN,Cluster 18632,3,train
2,CC1C(=O)N(C)C1C.O|MSIQHFRVALIPFFAAFCLPVFAHPETL...,3.5.2.6,P62593-c0_m0,SGPPFPMLSTGKVLCGCVLRSMAGKCQLGRVIMYMQNDWYEYSPTE...,[],NaN,Cluster 18632,3,train
3,CC1C(=O)N(C)C1C.O|MSIQHFRVALIPFFAAFCLPVFAHPETL...,3.5.2.6,P62593-c0_m10,FRPEEFPMMSTFKFLLCNAVLSCVDAGQEQLGRVISQILEEQMPVR...,[],NaN,Cluster 18632,3,train
4,CC1C(=O)N(C)C1C.O|MSIQHFRVALIPFFAAFCLPVFAHPETL...,3.5.2.6,P62593-c0_m19,FPEERFPMSLKVLLCGAVLSRVDAGQEEGRRIYYSQDLVEYSVTDK...,[],NaN,Cluster 18632,3,train
...,...,...,...,...,...,...,...,...,...
90,CP(=O)([O-])OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[...,6.1.1.20,P27001,MLEEALAAIQNARDLEELKALKARYLGKKGLLTQEMKGLSALPLEE...,"[[149], [178], [204], [218], [261], [314]]",NaN,Cluster 13284,6,test
91,NC(=O)CC[C@H]([NH3+])C(=O)[O-].Nc1ncnc2c1ncn2[...,6.3.5.4,P22106,MCSIFGVFDIKTDAVELRKKALELSRLMRHRGPDWSGIYASDNAIL...,"[[2], [51], [75], [76], [322], [325]]",NaN,Cluster 1460,6,test
92,C[C@H]1O[C@H](COP(C)(=O)[O-])[C@@H](OP(=O)([O-...,6.5.1.4,P46849,MKRMIALDGAQGEGGGQILRSALSLSMITGQPFTITSIRAGRAKPG...,"[[13], [308]]",NaN,Cluster 14477,6,test
93,C[C@@H](O)C(=O)[O-].C[C@@H]([NH3+])C(=O)[O-].N...,6.1.2.1,P25051,MNRIKVAILFGGCSEEHDVSVKSAIEIAANINKEKYEPLYIGITKS...,"[[19], [22], [99], [244], [250], [290], [311],...",NaN,Cluster 13982,6,test


In [4]:
rfdiffusion_data = ['O87172', 'P00655', 'P11586', 'P21164', 'P0A6L0', 'Q96C23']

In [5]:
end_aug_dataset['alphafolddb-id'].isin(rfdiffusion_data).sum()

1

In [6]:
end_aug_dataset.loc[end_aug_dataset['alphafolddb-id'].isin(rfdiffusion_data)]

,reaction,ec,alphafolddb-id,aa_sequence,site_labels,site_types,cluster,ec_level1,dataset_flag
27,OC[C@H]1OC(O)[C@H](O)[C@@H](O)[C@@H]1O.OC[C@H]...,2.4.1.25,O87172,MELPRAFGLLLHPTSLPGPYGVGVLGREARDFLRFLKEAGGRYWQV...,"[[293], [340], [395]]",NaN,Cluster 3762,2,test


In [4]:

nb_workers = 10
pandarallel.initialize(nb_workers=nb_workers, progress_bar=True)

end_aug_dataset['aa_sequence_calculated'] = end_aug_dataset['alphafolddb-id'].parallel_apply(
            lambda x: get_sequence(fixed_generated_structure_dir, x))


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
print((end_aug_dataset['aa_sequence_calculated'] != end_aug_dataset['aa_sequence']).sum())
print((end_aug_dataset['aa_sequence_calculated'] != '').sum())
print(len(end_aug_dataset['aa_sequence_calculated']))

5192
4910
5473


In [ ]:
# end_aug_dataset.to_csv('end_aug_dataset_mcsa.csv', index=False)

In [ ]:
from collections import defaultdict
not_same_aa = defaultdict(int)
not_same_aa_id = defaultdict(list)
length_not_same = 0
for aa_sequence, aa_sequence_calculated, structure_id in zip(end_aug_dataset['aa_sequence'].tolist(), end_aug_dataset['aa_sequence_calculated'].tolist(), end_aug_dataset['alphafolddb-id']):
    
    if len(aa_sequence) != len(aa_sequence_calculated):
        length_not_same += 1
        continue
    
    aa_sequence_np = np.array(list(aa_sequence))
    aa_sequence_calculated_np = np.array(list(aa_sequence_calculated))
    
    not_same = aa_sequence_np != aa_sequence_calculated_np
    org_aa = aa_sequence_np[not_same]
    cal_aa = aa_sequence_calculated_np[not_same]
    not_same_index = np.argwhere(not_same)
    
    for o_a, c_a, aa_index in zip(org_aa.tolist(), cal_aa.tolist(), not_same_index.tolist()):
        not_same_aa[(o_a, c_a)] += 1
        not_same_aa_id[(o_a, c_a)].append((structure_id, aa_index))

In [ ]:
end_aug_dataset.loc[end_aug_dataset['alphafolddb-id'] == 'P81701-c0_m12']

,reaction,ec,alphafolddb-id,aa_sequence,site_labels,site_types,cluster,ec_level1,dataset_flag,aa_sequence_calculated
2947,CSc1ccccc1.OO|QTCSTSDDADDPTPPNERDDEAFASRVAAAKR...,1.11.1.18,P81701-c0_m12,WLAQRWYQKWQVHRWARPEALGGTLHLTFKGELNADFDLPLLENAE...,[],NaN,Cluster 1320,1,train,WLAQRWYQKWQVXRWARPEALGGTLXLTFKGELNADFDLPLLENAE...


In [ ]:
[x for x in not_same_aa_id.items() if x[0][-1]!='X']

[(('E', 'Q'), [('P00488', [651])])]

In [ ]:
len(not_same_aa_id)

2

In [ ]:
not_same_aa_id[('E', 'Q')]

[('P00488', [651])]

In [ ]:
len(not_same_aa_id[('H', 'X')])

28481

In [ ]:
end_aug_test_dataset = end_aug_dataset.loc[end_aug_dataset['dataset_flag']=='test']
print((end_aug_test_dataset['aa_sequence_calculated'] != end_aug_test_dataset['aa_sequence']).sum())
print((end_aug_test_dataset['aa_sequence_calculated'] != '').sum())
print(len(end_aug_test_dataset['aa_sequence_calculated']))

95
81
95


In [ ]:
# # 测试一个映射
# index = 0
# need_fix_end_aug_dataset = end_aug_dataset.loc[(end_aug_dataset['aa_sequence_calculated'] != '') & (end_aug_test_dataset['aa_sequence_calculated'] != end_aug_test_dataset['aa_sequence'])]

# map_active_site_for_one_return_list()